In [1]:
import os
import pickle
import numpy as np
from scipy.io import loadmat

import scfw.cov_estimation as ce
from scfw.frank_wolfe import frank_wolfe
from scfw.scopt import scopt

In [2]:
data_folder = '../data/'
problem_name = 'Lymph.mat'
results = {problem_name: {}}
results_folder = './results/'
results_file = os.path.join(results_folder, 'cov_estimation', problem_name + '.pckl')
if os.path.isfile(results_file):
    with open(results_file, "rb") as f:
        try:
            results = pickle.load(f)
        except Exception:
            pass 

In [3]:
def sample_l1_point(r, ndim):
    vec = np.random.rand(ndim,)
    vec /= np.linalg.norm(vec, ord=1)
    return vec

In [19]:
# load data
data = loadmat(os.path.join(data_folder, problem_name))
    
A = data['S']
p = A.shape[0]
r = 50
X_0 = r * np.eye(p) / p
#X_0 = np.diag(sample_l1_point(r, p))

## Parameters initialization

In [20]:
#first set of parameters
Mf = 2
nu = 3
#running parameters

terminate_tol = 1e-10

FW_params={
    'iter_FW': 100,
    'line_search_tol': 1e-10,
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'estimate', #estimate L
    'use_two_phase': True,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol': 1e-2,
    'conj_grad_iter': 100,
}

## Functions

In [21]:
func_x = lambda X: ce.cov_val(A, X)
func_beta = lambda X, S, beta, extra_param, extra_param_s: ce.cov_val(A, (1 - beta) * X + beta * S)
grad_x = lambda X, inv_X: ce.cov_grad(A, X, inv_X)
grad_beta = lambda X, S, beta, inv_X: ce.cov_grad(A, (1 - beta) * X + beta * S) # TODO: add inverse
hess_mult_x = lambda S, inv_X: ce.cov_hess_mult(S, inv_X)
hess_mult_vec_x = lambda S, trace_sum: ce.cov_hess_mult_vec(S, inv_X)
extra_func = lambda X: np.linalg.eigvalsh(X)
linear_oracle = lambda grad: ce.linear_oracle(grad, r)
prox_func = lambda X, L: ce.proj_map(X) #used for SCOPT

## Run FW

In [25]:
#run_alpha_policies = ["standard", "line_search", "icml", "backtracking"]
run_alpha_policies = ["standard"]

for policy in run_alpha_policies:
    x, alpha_hist, Gap_hist, Q_hist, time_hist  = frank_wolfe(func_x,
                       func_beta,                                       
                       grad_x,
                       grad_beta,
                       hess_mult_x,
                       extra_func,                                                    
                       Mf,
                       nu,
                       linear_oracle,                                                    
                       X_0,
                       FW_params,
                       #hess=hess_x, 
                       #lloo_oracle=llo_oracle,                                                 
                       alpha_policy=policy,                                                    
                       eps=terminate_tol, 
                       print_every=1, 
                       debug_info=False)
    
    results[problem_name][policy] = {
        'x': x,
        'alpha_hist': alpha_hist,
        'Gap_hist': Gap_hist,
        'Q_hist': Q_hist,
        'time_hist': time_hist,
    }

********* Algorithm starts *********
f: 1495.782065 Gap: -1.705303e-13
iter = 1, stepsize = 0.5, criterion = 1.0, upper_bound=1495.7820650527729, lower_bound=1495.782065052773, real_Gap=-2.2737367544323206e-13
f: 1896.282733 Gap: 5.870000e+02
iter = 2, stepsize = 0.4, criterion = 0.8919983278847288, upper_bound=1495.7820650527729, lower_bound=1495.782065052773, real_Gap=-2.2737367544323206e-13
f: 2189.473390 Gap: 1.369667e+03
iter = 3, stepsize = 0.3333333333333333, criterion = 0.7440185574736169, upper_bound=1495.7820650527729, lower_bound=1495.782065052773, real_Gap=-2.2737367544323206e-13
f: 2420.594537 Gap: 2.348000e+03
iter = 4, stepsize = 0.2857142857142857, criterion = 0.6691180366054715, upper_bound=1495.7820650527729, lower_bound=1495.782065052773, real_Gap=-2.2737367544323206e-13
f: 2611.034716 Gap: 3.522000e+03
iter = 5, stepsize = 0.25, criterion = 0.6181403338094523, upper_bound=1495.7820650527729, lower_bound=1495.782065052773, real_Gap=-2.2737367544323206e-13
f: 2772.681

KeyboardInterrupt: 

In [8]:
func_x(X_0)

(nan, array([[-1469.34901145,    -0.        ,    -0.        , ...,
            -0.        ,    -0.        ,    -0.        ],
        [   -0.        ,  -221.87797209,    -0.        , ...,
            -0.        ,    -0.        ,    -0.        ],
        [   -0.        ,    -0.        ,  -457.93696046, ...,
            -0.        ,    -0.        ,    -0.        ],
        ...,
        [   -0.        ,    -0.        ,    -0.        , ...,
         -1842.4513232 ,    -0.        ,    -0.        ],
        [   -0.        ,    -0.        ,    -0.        , ...,
            -0.        , -4522.01861138,    -0.        ],
        [   -0.        ,    -0.        ,    -0.        , ...,
            -0.        ,    -0.        ,  -392.73802279]]))

In [11]:
1 / p

0.0017035775127768314

In [10]:
np.diag(X_0)

array([-6.80573500e-04, -4.50698188e-03, -2.18370668e-03,  3.01983470e-03,
       -3.27882868e-03,  2.29253846e-04,  1.13508338e-03,  2.33774830e-04,
       -1.42513497e-03,  3.15209217e-03,  1.54156840e-03, -3.21425694e-03,
       -1.82570357e-03,  3.85337360e-03,  2.69623128e-03, -2.98060194e-04,
       -2.01372044e-03,  7.15853313e-04,  2.05461860e-03,  1.06456813e-03,
       -3.22827541e-03,  2.97544840e-03,  2.08333804e-03,  4.91983295e-04,
       -1.83512940e-03,  9.28407354e-04, -1.00835638e-03, -1.06009277e-03,
        7.72906916e-04,  1.33349870e-03, -5.95158923e-04,  6.47452436e-04,
        2.30969871e-03,  8.37492208e-04, -1.09729103e-03, -4.17433867e-03,
        1.61838037e-03,  1.76493277e-03, -1.45301270e-03,  1.59489027e-03,
        2.41891281e-05,  1.75209222e-03, -1.86087790e-03,  3.02945582e-03,
        1.93100392e-03,  2.38266545e-03,  7.62657189e-05, -2.27167719e-03,
       -2.69730218e-03,  1.46065197e-03,  1.47014870e-03, -1.43536286e-03,
        6.43824879e-04, -

In [12]:
S = linear_oracle(grad)

In [13]:
x_nxt = X_0 + 2/3 * (S - X_0)

In [16]:
np.sum(S)

-1.0

In [19]:
x_nxt

array([[0.00056786, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00056786, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00056786, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00056786, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00056786,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00056786]])

In [22]:
A = np.array([[1, -3], [-4, 3]])

In [13]:
x = X_0
f_vals = []
for i in range(100):
    f, inv_X = func_x(x)
    x = x - 0.01 * grad_x(x, inv_X)
    f_vals.append(f)

In [14]:
f_vals

[3743.1395692390934,
 2253.9285299320827,
 2101.179286305169,
 1948.5753972224338,
 1796.1892120512784,
 1644.1986204095851,
 1493.445091643542,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]